Load packages.

In [1]:
import itertools
import json
import numpy as np
import os
import pandas as pd
import pysam

from basenji import dataset, dna_io, seqnn
from scipy import stats

Define some core functions that are used for prediction from sequence.

In [2]:
def loadAkita():
    os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

    import tensorflow as tf
    if tf.__version__[0] == '1':
        tf.compat.v1.enable_eager_execution()

    with open('/wynton/group/capra/projects/pan_3d_genome/data/model/params.json') as params_file:
        params = json.load(params_file)
        params_model = params['model']
        params_train = params['train']
        
    global seqnn_model
    seqnn_model = seqnn.SeqNN(params_model)
    seqnn_model.restore('/wynton/group/capra/projects/pan_3d_genome/data/model/model_best.h5')

In [3]:
loadAkita()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence (InputLayer)           [(None, 1048576, 4)] 0                                            
__________________________________________________________________________________________________
stochastic_reverse_complement ( ((None, 1048576, 4), 0           sequence[0][0]                   
__________________________________________________________________________________________________
stochastic_shift (StochasticShi (None, 1048576, 4)   0           stochastic_reverse_complement[0][
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 1048576, 4)   0           stochastic_shift[0][0]           
____________________________________________________________________________________________

In [4]:
def runAkitaPreds(seq):
    if len(seq) != 2**20: raise ValueError('len(seq) != seq_length')
    seq_1hot = dna_io.dna_1hot(seq)
    test_pred_from_seq = seqnn_model.model.predict(np.expand_dims(seq_1hot,0))
    return test_pred_from_seq

In [5]:
def comparePreds(ref, alt):
    mse = np.mean(np.square(ref - alt))
    spearman = stats.spearmanr(ref, alt)[0]
    divergence = 1 - spearman
    return (mse, divergence)

Test for in silico mutagenesis SNV script.

In [18]:
def get_contact_maps():
    with open(f'/wynton/group/capra/projects/pan_3d_genome/data/predictions/reference/3d_predictions_HFF_{chr}.txt', 'r') as ref_file:
        for line in ref_file:
            ref_line = line.split('\t')
            if ref_line[0] == chr and ref_line[1] == window_start:
                print(ref_line[0])
                print(ref_line[1])
                ref_pred_hff = list(map(float,ref_line[2:]))

    ref_fasta_open = pysam.Fastafile('/wynton/group/capra/projects/pan_3d_genome/data/reference_fasta/filtered_panTro6.fa')
    ref_seq = ref_fasta_open.fetch(chr, int(window_start), int(window_start)+2**20).upper()
    pos_in_window = (int(variant_pos)-1)-int(window_start)
    alt_seq = ref_seq[0:pos_in_window] + alt_allele + ref_seq[pos_in_window+1:]
    alt_pred = runAkitaPreds(alt_seq)
    alt_pred_hff = alt_pred[:,:,0][0]
    return ref_pred_hff, alt_pred_hff

In [14]:
def in_silico_mutagenesis_SNV_test(chromosome,pos,alt,target_window):
    global chr
    chr = chromosome
    
    global variant_pos
    variant_pos = pos
    
    global alt_allele
    alt_allele = alt
    
    global window
    window = target_window
    
    global window_start
    window_start = window.split('_')[1]
    
    ref_prediction, alt_prediction = get_contact_maps()
    mse, divergence = comparePreds(np.array(ref_prediction), np.array(alt_prediction))
    return mse, divergence

In [8]:
in_silico_mutagenesis_SNV_test('chr13',44760355,'C','chr13_44564480')

chr13
44564480


(0.0004109143222303805, 0.014099135528463869)

In [9]:
in_silico_mutagenesis_SNV_test('chr13',45090113,'T','chr13_45088768')

chr13
45088768


(2.6360405091583755e-11, 1.1632317331589093e-09)

In [19]:
in_silico_mutagenesis_SNV_test('chr13',3145764,'T','chr13_3145728')

chr13
3145728


(1.0089372026745314e-10, 3.3496874163319035e-09)

Test for in silico mutagenesis inversions script.

In [6]:
def get_alt_map():
    ref_fasta_open = pysam.Fastafile('/wynton/group/capra/projects/pan_3d_genome/data/reference_fasta/filtered_panTro6.fa')
    ref_seq = ref_fasta_open.fetch(chr, int(window_start), int(window_start)+2**20).upper()

    inv_start_in_window = (int(inv_start)-int(window_start))
    inv_end_in_window = (int(inv_end)-int(window_start))
    seq_to_invert = ref_seq[inv_start_in_window:inv_end_in_window]
    inverted_seq = seq_to_invert[::-1]
    alt_seq = ref_seq[0:inv_start_in_window] + inverted_seq + ref_seq[inv_end_in_window:]
    alt_pred = runAkitaPreds(alt_seq)
    alt_pred = alt_pred[:,:,0][0]
    return alt_pred

In [16]:
def load_ref_map():
    with open(f'/wynton/group/capra/projects/pan_3d_genome/data/predictions/reference/3d_predictions_HFF_{chr}.txt', 'r') as ref_file:
        for line in ref_file:
            if window_start in line:
                ref_line = line.split('\t')
                ref_pred = list(map(float,ref_line[2:]))
    return ref_pred

In [23]:
df = []

with open(f'../../data/Porubsky_et_al_2020/example.bed', 'r') as input, open(f'results.txt', 'w') as out:
    for line in input:
        line = line.split('\t')
            
        global chr
        chr = line[0]

        global inv_start
        inv_start = line[1]

        global inv_end
        inv_end = line[2]

        global window_start
        window_start = line[3].strip()

        alt_pred = get_alt_map()
        print(window_start)
        ref_pred = load_ref_map()
        mse, pearson, spearman = compare3Dmaps(ref_pred, alt_pred)

        df.append({'chr':chr,'start':int(inv_start),'end':int(inv_end),'window':int(window_start),'mse':mse,'1-pearson':pearson,'1-spearman':spearman})
        print(f'Akita prediction for window {window_start} with inversion {chr}: {inv_start}-{inv_end} completed.')

        if spearman >= 0.01:
            with open(f'/wynton/group/capra/projects/pan_3d_genome/data/Porubsky_et_al_2020/inversion_predictions/{chr}_{inv_start}_{window_start}.txt', 'w') as pred_out:
                pred_out.write(chr + "\t" + str(inv_start) + "\t" + "\t".join([str(x) for x in alt_pred]))

    df = pd.DataFrame(df)[['chr','start','end','window','mse','1-pearson','1-spearman']]
    df.to_csv(out, sep='\t', index=False)

185597952
Akita prediction for window 185597952 with inversion chr1: 186157349-186164563 completed.


In [18]:
spearman

0.02302839970757009

In [19]:
ref_pred

[0.042321533,
 0.04950109,
 0.049550265,
 0.051678896,
 0.05693528,
 0.06291446,
 0.06928441,
 0.07436684,
 0.079781264,
 0.08680591,
 0.09564158,
 0.10340151,
 0.10618511,
 0.10733941,
 0.11223081,
 0.112941414,
 0.11043611,
 0.1095058,
 0.10626182,
 0.103426665,
 0.0943093,
 0.08957437,
 0.0859817,
 0.08232996,
 0.07758829,
 0.07581434,
 0.080748945,
 0.10042378,
 0.118136376,
 0.13834044,
 0.15040538,
 0.15905139,
 0.16446969,
 0.16588488,
 0.17167863,
 0.1741775,
 0.17216137,
 0.16942212,
 0.1533142,
 0.13449094,
 0.111514956,
 0.0961034,
 0.08350113,
 0.07733294,
 0.07259634,
 0.042252213,
 -0.009914488,
 -0.07685408,
 -0.12591603,
 -0.16656533,
 -0.19786733,
 -0.21263045,
 -0.22486229,
 -0.23924942,
 -0.25371313,
 -0.2661047,
 -0.2680828,
 -0.2724446,
 -0.27890155,
 -0.28602973,
 -0.28879505,
 -0.2874979,
 -0.2977839,
 -0.30759743,
 -0.31771496,
 -0.3211323,
 -0.33061904,
 -0.3331777,
 -0.33702007,
 -0.33685333,
 -0.33755425,
 -0.34171572,
 -0.33781585,
 -0.34487364,
 -0.34706503